"""
Mission Log: Starship Odyssey - Uncharted Terrestrial Planet Survey

A faint, long-dormant beacon signal, fragmented but unmistakably artificial, has shattered the deep space silence. 
After weeks of high-resolution triangulation, we've managed to zero in on its possible origin: a volatile 50x50 kilometer sector 
of an uncharted terrestrial planet. This isn't a simple 'land or water' scenario; our preliminary atmospheric scans indicate 
a chaotic tapestry of geological anomalies, treacherous zones, and unknown energy signatures.

Our objective is not to land. Odyssey is a reconnaissance vessel, and the terrain stability is unknown. 
Instead, our advanced orbital scanners must analyze each square kilometer of this 50x50 grid. 
Every block holds an 'object' – a unique geological formation, an atmospheric phenomenon, or a physical obstacle. We must:

1.  **Determine Navigable Terrain:** Based on each object's detailed geological properties (rock, silicate, water, carbon density, composition) a
nd structural stability, our system must classify each square kilometer as 'traversable' (land for our survey drones) or 'impassable' (water/hazard). 
There's no direct 'is_land' flag; it's a complex scientific analysis.

2.  **Locate the Beacon Source:** Within this complex terrain, somewhere is the source of the beacon. We don't know if it's an ancient base, 
a derelict crew ship, or something entirely alien.

3.  **Uncover the Beacon's Secret:** The moment we pinpoint the Beacon Source, its true nature (its 'encoded discovery data') must be extracted a
nd reported. This information, hidden within the object itself, is the core of our mission.

This is Odyssey's most critical mission yet: to navigate the unknown, analyze the unanalyzable, 
and unveil the secret of the long-lost beacon. Failure is not an option.
"""

In [44]:
import random
import numpy
import sys
import time
import os
import psutil
from collections import deque

numpy.set_printoptions(threshold=sys.maxsize)
numpy.set_printoptions(linewidth=300)

# --- Global Constants for Traversability Analysis ---
MIN_STABILITY_FOR_TRAVERSAL = 0.5              # Terrain must have stability >= 0.5 to be traversable
MAX_WATER_COMPOSITION_FOR_TRAVERSAL = 0.3      # Terrain must have water <= 0.3 to be traversable
MIN_SOLID_COMPOSITION_FOR_TRAVERSAL = 0.4      # Terrain's (rock + silicates) must be <= 0.4 to be traversable
# --- End Global Constants ---
TRAVERSE_LOGIX = deque([[-1,0],[0,1],[1,0],[0,-1]])
PLANET_GRID_SIZE = 10

# --- Operations Handlers ---
toTraverse = deque()
scannedGrid = numpy.array([PLANET_GRID_SIZE,PLANET_GRID_SIZE])

class TerrainObject:
    """Base class for all terrain features, requiring geological analysis for traversability."""
    def __init__(self, name, description,
                 geological_properties=None, stability=0.0, is_obstacle=False):
        self.name = name
        self.description = description
        self.geological_properties = geological_properties if geological_properties is not None else {
            'rock': 0.0, 'silicates': 0.0, 'water': 1.0, 'carbon': 0.0
        }
        self.stability = stability # 0.0 (unstable) to 1.0 (very stable)
        self.is_obstacle = is_obstacle # Direct obstacles are always impassable
        self.details = f"Details: {description}" # Hidden info for scanning

    def is_traversable(self):
        # Condition 1: Direct obstacle check
        if self.is_obstacle:
            return False

        # Condition 2: Stability check
        if self.stability < MIN_STABILITY_FOR_TRAVERSAL: # e.g., < 0.5
            return False

        # Condition 3: Water composition check
        if self.geological_properties.get('water', 0.0) > MAX_WATER_COMPOSITION_FOR_TRAVERSAL: # e.g., > 0.3
            return False

        # Condition 4: Solid composition (rock + silicates) check
        # IMPORTANT: Your logic makes 'too much solid' (i.e., > 0.4) IMPASSABLE.
        # This means for traversable terrain, solid_comp MUST be <= 0.4.
        solid_comp = self.geological_properties.get('rock', 0.0) + self.geological_properties.get('silicates', 0.0)
        if solid_comp > MIN_SOLID_COMPOSITION_FOR_TRAVERSAL: # e.g., > 0.4
            return False
        
        # If all checks pass, the terrain is traversable
        return True

    def __repr__(self):
        status = "Traversable" if self.is_traversable() else "Impassable (aerial transit required)"
        return f"{self.name}({status}, Stab:{self.stability:.1f})"

# --- Reliably Traversable Terrain ---
class FlatGround(TerrainObject):
    """Basic, easily traversable ground."""
    def __init__(self):
        super().__init__("Flat Ground", "Solid, even, and clear ground.",
                         # Properties configured to consistently be traversable:
                         geological_properties={
                             'rock': random.uniform(0.10, 0.20),      # Sum with silicates is <= 0.4
                             'silicates': random.uniform(0.05, 0.10), # Total solid: 0.15 - 0.30 (<= 0.4 OK)
                             'water': random.uniform(0.0, 0.10),      # Water: 0.0 - 0.10 (<= 0.3 OK)
                             'carbon': random.uniform(0.0, 0.05)
                         },
                         stability=random.uniform(0.8, 1.0),         # Stability: 0.8 - 1.0 (>= 0.5 OK)
                         is_obstacle=False)                           # Not a direct obstacle (OK)
    # VERDICT: This type will consistently be TRAVERSABLE.

class BeaconSource(TerrainObject):
    """The mysterious beacon emitting the signal."""
    def __init__(self, encoded_data, name="Beacon Source"):
        super().__init__(name, "The origin of the signal.",
                         # Adjusted geological properties to make it traversable with current constants.
                         # Its solid composition (rock+silicates) must be <= MIN_SOLID_COMPOSITION_FOR_TRAVERSAL (0.4).
                         geological_properties={
                             'rock': random.uniform(0.10, 0.20),      # Sum with silicates is <= 0.4
                             'silicates': random.uniform(0.05, 0.10), # Total solid: 0.15 - 0.30 (<= 0.4 OK)
                             'water': random.uniform(0.0, 0.05),      # Water: 0.0 - 0.05 (<= 0.3 OK)
                             'carbon': random.uniform(0.0, 0.05)
                         },
                         stability=random.uniform(0.9, 1.0),         # Stability: 0.9 - 1.0 (>= 0.5 OK)
                         is_obstacle=False)                           # Not a direct obstacle (OK)
        self.encoded_discovery_data = encoded_data

    def __repr__(self):
        status = "Traversable" if self.is_traversable() else "IMPASSABLE (ERROR)"
        return f"{self.name}({status}, ID:{self.encoded_discovery_data})"
    # VERDICT: This type will now be TRAVERSABLE.

# --- Non-Traversable / Obstacle Terrain Types ---
class EmptySpace(TerrainObject):
    """Vast, deep space / unchartable abyss."""
    def __init__(self):
        super().__init__("Ocean Abyss", "A void where nothing can exist.",
                         geological_properties={'rock': 0, 'silicates': 0, 'water': 1.0, 'carbon': 0}, # Water 1.0 (> 0.3 FAIL)
                         stability=0.0,                                                             # Stability 0.0 (< 0.5 FAIL)
                         is_obstacle=False)
    # VERDICT: NOT traversable (fails water and stability checks).

class RockyTerrain(TerrainObject):
    """Solid, high-density rock formations."""
    def __init__(self, name="Rocky Terrain"):
        super().__init__(name, "Dense, ancient rock formations.",
                         geological_properties={
                             'rock': random.uniform(0.6, 0.9),       # Sum with silicates > 0.4
                             'silicates': random.uniform(0.05, 0.2), # Total solid: 0.65 - 1.10 (> 0.4 FAIL)
                             'water': random.uniform(0, 0.1),
                             'carbon': random.uniform(0, 0.05)
                         },
                         stability=random.uniform(0.7, 1.0),
                         is_obstacle=False)
    # VERDICT: NOT traversable (fails solid composition check).

class VolcanicCrater(TerrainObject):
    """Highly unstable, molten areas."""
    def __init__(self):
        super().__init__("Volcanic Crater", "A bubbling, molten pit.",
                         geological_properties={'rock': 0.1, 'silicates': 0.1, 'water': 0.4, 'carbon': 0.0},
                         stability=random.uniform(0.0, 0.4), # Stability: 0.0 - 0.4 (< 0.5 FAIL)
                         is_obstacle=False)
    # VERDICT: NOT traversable (fails stability check).

class HydrothermalVent(TerrainObject):
    """Underwater/subsurface hot vents, too watery/unstable."""
    def __init__(self):
        super().__init__("Hydrothermal Vent", "Geothermal activity creating hot, unstable water flows.",
                         geological_properties={'rock': 0.05, 'silicates': 0.05, 'water': random.uniform(0.7, 0.9), 'carbon': 0.1}, # Water: 0.7 - 0.9 (> 0.3 FAIL)
                         stability=random.uniform(0.1, 0.3),                                                                       # Stability: 0.1 - 0.3 (< 0.5 FAIL)
                         is_obstacle=False)
    # VERDICT: NOT traversable (fails water and stability checks).

# --- Explicit Obstacle Types (Always Impassable by definition) ---
class DenseFog(TerrainObject):
    """Thick atmospheric conditions, blocking all sensors and traversal."""
    def __init__(self):
        super().__init__("Dense Fog", "An atmospheric anomaly, impenetrable.",
                         geological_properties={'rock': 0, 'silicates': 0, 'water': 1, 'carbon': 0},
                         stability=0.0,
                         is_obstacle=True) # Direct obstacle (FAIL)
    # VERDICT: NOT traversable.

class MagneticAnomaly(TerrainObject):
    """Intense magnetic fields disrupting all systems."""
    def __init__(self):
        super().__init__("Magnetic Anomaly", "A localized magnetic field, disrupts all electronics.",
                         geological_properties={},
                         stability=0.0,
                         is_obstacle=True) # Direct obstacle (FAIL)
    # VERDICT: NOT traversable.

class EnergyBarrier(TerrainObject):
    """An unknown, static energy field."""
    def __init__(self):
        super().__init__("Energy Barrier", "A shimmering, impassable energy wall.",
                         geological_properties={},
                         stability=0.0,
                         is_obstacle=True) # Direct obstacle (FAIL)
    # VERDICT: NOT traversable.
    
# Triangulating baecon's last know possible grid sector
def generateGrid(PLANET_GRID_SIZE):

    gridSector_Delta = numpy.full((PLANET_GRID_SIZE, PLANET_GRID_SIZE), EmptySpace(), dtype=object)

    # Define coordinates for the beacon and a starting point
    beacon_coords = (random.randint(0, PLANET_GRID_SIZE - 1), random.randint(0, PLANET_GRID_SIZE - 1))
    start_coords = (random.randint(0, PLANET_GRID_SIZE - 1), random.randint(0, PLANET_GRID_SIZE - 1))

    # Ensure start and beacon are not the same (optional)
    while start_coords == beacon_coords:
        start_coords = (random.randint(0, PLANET_GRID_SIZE - 1), random.randint(0, PLANET_GRID_SIZE - 1))

    for r in range(PLANET_GRID_SIZE):
        for c in range(PLANET_GRID_SIZE):
            # Always place beacon and start first
            if (r, c) == beacon_coords:
                gridSector_Delta[r, c] = BeaconSource("alpha_signal_data")
            elif (r, c) == start_coords:
                gridSector_Delta[r, c] = FlatGround() # Ensure start is traversable
            else:
                # Prioritize placing various hazards
                hazard_roll = random.random() # Roll the dice for hazard type

                if hazard_roll < 0.20: # 20% chance for Rocky Terrain (Impassable)
                    gridSector_Delta[r, c] = RockyTerrain()
                elif hazard_roll < 0.35: # Next 15% (total 35%) chance for Ocean Abyss (Impassable)
                    gridSector_Delta[r, c] = EmptySpace() # This is your "Ocean Abyss"
                elif hazard_roll < 0.45: # Next 10% (total 45%) chance for Volcanic Crater (Impassable)
                    gridSector_Delta[r, c] = VolcanicCrater()
                elif hazard_roll < 0.55: # Next 10% (total 55%) chance for Hydrothermal Vent (Impassable)
                    gridSector_Delta[r, c] = HydrothermalVent()
                elif hazard_roll < 0.65: # Next 10% (total 65%) chance for Dense Fog (Impassable)
                    gridSector_Delta[r, c] = DenseFog()
                elif hazard_roll < 0.70: # Next 5% (total 70%) chance for Magnetic Anomaly (Impassable)
                    gridSector_Delta[r, c] = MagneticAnomaly()
                elif hazard_roll < 0.75: # Next 5% (total 75%) chance for Energy Barrier (Impassable)
                    gridSector_Delta[r, c] = EnergyBarrier()
                else: # The remaining percentage (25% in this example) will be FlatGround
                    gridSector_Delta[r, c] = FlatGround()

    return gridSector_Delta

start_time = time.perf_counter()
process = psutil.Process(os.getpid())
before_memory = process.memory_info().rss
gridSector_Delta  = generateGrid(PLANET_GRID_SIZE)
end_time = time.perf_counter()
elapsed_time = end_time - start_time
after_memory = process.memory_info().rss
memory_used = after_memory - before_memory
print(f"Elapsed time: {elapsed_time:.2f} seconds")
print(f"Memory used: {memory_used/(1024*1024):.2f} MB ({process.memory_percent():.2f} %)")

Elapsed time: 0.00 seconds
Memory used: 0.00 MB (0.61 %)


In [42]:
def display_grid(planet_grid, rows_to_display=None, cols_to_display=None):
    """
    Prints a simplified map-like representation of the planet grid.

    Args:
        planet_grid (numpy.ndarray): The 2D NumPy array containing TerrainObject instances.
        rows_to_display (tuple, optional): A (start_row, end_row) tuple to display a sub-section.
                                            Defaults to None (display all rows).
        cols_to_display (tuple, optional): A (start_col, end_col) tuple to display a sub-section.
                                            Defaults to None (display all columns).
    """
    grid_rows, grid_cols = planet_grid.shape

    start_r = rows_to_display[0] if rows_to_display else 0
    end_r = rows_to_display[1] if rows_to_display else grid_rows
    
    start_c = cols_to_display[0] if cols_to_display else 0
    end_c = cols_to_display[1] if cols_to_display else grid_cols

    print(f"\n--- Planet Grid Map ({start_r}:{end_r}x{start_c}:{end_c}) ---")
    for r in range(start_r, end_r):
        for c in range(start_c, end_c):
            obj = planet_grid[r, c]
            
            # Decide on a single character representation for each object type
            char = '?' # Default for unknown or unhandled types

            if isinstance(obj, BeaconSource):
                char = 'B' # Beacon
            elif isinstance(obj, RockyTerrain):
                char = 'R' # Rocky Terrain
            elif isinstance(obj, VolcanicCrater):
                char = 'V' # Volcanic Crater
            elif isinstance(obj, HydrothermalVent):
                char = 'H' # Hydrothermal Vent
            elif isinstance(obj, DenseFog):
                char = 'F' # Dense Fog
            elif isinstance(obj, MagneticAnomaly):
                char = 'M' # Magnetic Anomaly
            elif isinstance(obj, EnergyBarrier):
                char = 'E' # Energy Barrier
            elif isinstance(obj, EmptySpace):
                char = '.' # Empty space / deep abyss
            
            # Print the character without a newline
            print(char, end=' ')
        print() # Print a newline after each row
    print("------------------------------------------")
display_grid(gridSector_Delta)


--- Planet Grid Map (0:10x0:10) ---
R R . ? H ? R R R ? 
R R V . ? R ? V F . 
V F . V V ? R . . ? 
R . H . ? . ? ? ? R 
F R R F R . V . R ? 
H M . ? . F ? F R ? 
M ? M R B H V ? V R 
H ? . H ? ? ? R ? ? 
. R ? F R R M ? R E 
. R R ? ? E . ? V R 
------------------------------------------


In [47]:
# ------------------------Traverse through the grid chart and record all blocks' properties and unravel hidden beacon's mystery -------------------
def chartingGridSector_Delta (gridSector_Delta):
    
    BOLD = "\033[1m"
    RESET ="\033[0m"
    print(f"{BOLD}chartedBlock {RESET}\n")

    for gridLat in range(len(gridSector_Delta)):
        for gridLong in range(len(gridSector_Delta)):
            currentBlock = gridSector_Delta[gridLat][gridLong]
            if(isinstance(currentBlock, BeaconSource)):
                print(currentBlock)
                
    return 

chartingGridSector_Delta(gridSector_Delta)

chartedBlock 

Beacon Source(Traversable, ID:alpha_signal_data)


In [2]:
import random
random.random()

0.7986029586715611